# **Exploratory Data Analysis**: FlipTop

## Question
----

The central question of this project revolves around FlipTop battle rap careers and their length.

> How long does a career in the Philippines' premier rap battle league FlipTop last?

Who rises to the top? What determines who stays relevant over the years?

Here I'm thinking survival (time-to-event) analysis.

Another question I'm curious about is this:
> Who has battled who? And what matchups haven't been done yet?

I'm picturing a graph here. I want to understand how dense or sparse the network of rap battles are in FlipTop.

Ideas:
- Emcees are nodes
- "Battled" as edges
- Weight of the edge is how many times they've battled?

## Data Wrangling
----

Data wrangled using [YouTube API](https://developers.google.com/youtube/v3).

In [1]:
import sys
import json
import pandas as pd
import numpy as np
import isodate
import re

Note: This data was scraped July 2, 2025.

In [2]:
# Load JSON data
with open("../data/videos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Preview
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02T13:23:54Z,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:48:58Z,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [3]:
# No need to run this again (already ran once and wrote to file)
sys.path.append("../scripts")

from emcee_scraper import scrape_names, write_names_to_csv
emcees = scrape_names()
write_names_to_csv(emcees, "../data/emcees.csv")

Scraped 20 names from page 1.
Scraped 20 names from page 2.
Scraped 20 names from page 3.
Scraped 20 names from page 4.
Scraped 20 names from page 5.
Scraped 20 names from page 6.
Scraped 20 names from page 7.
Scraped 20 names from page 8.
Scraped 7 names from page 9.
No emcees found on page 10. Stopping early.
Wrote 167 names to ../data/emcees.csv


## Data Cleaning
----

Tasks:

- convert `upload_date` to something more usable.
- convert `duration` to human readable (HH:MM:SS maybe?)
- Some shorts/round highlight moments were uploaded on the [videos playlist](https://www.youtube.com/@fliptopbattles/videos) of the channel so need to filter by duration, especially post-2020 (when YouTube Shorts became a thing).
- Only considering 1v1 battles that were judged (for now)
    - So remove:
        - Sound Check
        - Freestyle battles
        - Dos Por Dos (2v2)
        - Royal Rumble (1v1v1v1v1)
        - Those two 5v5 battles that happened only twice
        - Anygma Machine
        - Video Flyer
        - Announcement videos
        - Emcee interview videos

BIG TASKS

- Clean up `title` and create new column `battle_card` or something that contains only names e.g. Emcee1 vs Emcee2
- Webscrape [emcee page](https://www.fliptop.com.ph/emcees) to get a full and complete list of the emcees in FlipTop (according to their own website roster).
    - Good for later user of cross-referencing titles with emcee names.
- 

In [3]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02T13:23:54Z,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:48:58Z,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [4]:
# looking at desc
samp = df.sample(n=5, replace=False)
samp

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
1088,2jpJQTkLvUQ,FlipTop - Apoc vs Apekz,"FlipTop Metro Manila presents: Ahon 7, Day 2, ...",2017-02-18T13:43:35Z,3772309,PT27M54S,https://www.youtube.com/watch?v=2jpJQTkLvUQ,20782,4436,"[fliptop, apoc, apekz, ahon, ahon 7, san juan ..."
91,nF94vBh9Ac0,FlipTop - Harlem vs Katana,"FlipTop presents: Ahon 15, Day 1 @ The Tent, G...",2025-03-08T12:43:02Z,1099680,PT42M50S,https://www.youtube.com/watch?v=nF94vBh9Ac0,18351,2555,"[fliptop, flip top, flip top battles, fliptopb..."
1187,HFAUZ5w5osY,FlipTop - Thike vs Dhictah,"FlipTop Metro Manila presents: Ahon 6, Day 1, ...",2016-03-26T10:10:57Z,376898,PT21M49S,https://www.youtube.com/watch?v=HFAUZ5w5osY,1264,763,"[fliptop, thike, dhictah, fliptop new, fliptop..."
700,jtOnYVHQQUU,FlipTop - Pistolero vs Fukuda @ The FlipTop Fe...,The FlipTop Festival After Movie: https://yout...,2020-04-22T12:31:45Z,1936234,PT28M,https://www.youtube.com/watch?v=jtOnYVHQQUU,25980,3963,"[fliptop, flip top, flip top battles, fliptopb..."
1282,n0ltrzxrWPQ,FlipTop - Tipsy D vs J Skeelz,"FlipTop Metro Manila presents: Ahon 5, Day 2 @...",2015-03-04T14:22:58Z,4597803,PT31M17S,https://www.youtube.com/watch?v=n0ltrzxrWPQ,17907,3261,"[fliptop, tipsy d, j skeelz, fliptop latest, f..."


In [5]:
samp["description"]

1088    FlipTop Metro Manila presents: Ahon 7, Day 2, ...
91      FlipTop presents: Ahon 15, Day 1 @ The Tent, G...
1187    FlipTop Metro Manila presents: Ahon 6, Day 1, ...
700     The FlipTop Festival After Movie: https://yout...
1282    FlipTop Metro Manila presents: Ahon 5, Day 2 @...
Name: description, dtype: object

In [6]:
# Write multi-line descriptions to file for analysis
title_desc = samp[["title", "description"]]
records = title_desc.to_dict(orient="records")

with open("../data/sample_descriptions.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

In [7]:
with open("../data/sample_descriptions.json", "r", encoding="utf-8") as f:
    descriptions = json.load(f)

# Pick the first one (index 0) or whichever index you want
sample = descriptions[1]

# Print nicely
print("TITLE:\n", sample["title"])
print("\nDESCRIPTION:\n", sample["description"])

TITLE:
 FlipTop - Harlem vs Katana

DESCRIPTION:
 FlipTop presents: Ahon 15, Day 1 @ The Tent, Gatchalian Drive, Las Pinas City, Metro Manila, Philippines. Dec. 20-21, 2024. Filipino Conference Battle 
- HARLEM vs KATANA - 

Subscribe Here! http://bit.ly/fliptopsub
Check out our top videos! http://bit.ly/fliptopTopVideos

Facebook: https://www.facebook.com/fliptop.battleleague
Instagram: https://instagram.com/fliptopbattles
Twitter: https://twitter.com/FlipTop_Battles
Patreon: https://patreon.com/fliptop
For Sponsorship: Email partner@fliptop.com.ph 

#FlipTop #Ahon15 #Harlem #Katana

About fliptopbattles:
FlipTop Kru Corp. is a self-produced events and artist management company with its first product in the FlipTop Battle League. The FlipTop Battle League is the Philippines’ first premier – and the world’s most-viewed – rap battle league. It is popularly credited for the resurgence and widespread acceptance of hiphop culture in the Philippines since its inception in February 2010, and

Video descriptions often have good info on them for when and where the rap battle took place. Need to scrape these too!
- Let's put a pin on that right now. Do low-hanging fruit first before going for messy regex text extraction tasks.

In [8]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02T13:23:54Z,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:48:58Z,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [9]:
# Convert to upload_date to datetime objects
df["upload_date"] = pd.to_datetime(df["upload_date"], utc=True)

In [10]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [11]:
# Create duration_timedelta col
df["duration_timedelta"] = df["duration"].apply(isodate.parse_duration)

# Create duration_seconds col (for ez calculations down the line)
df["duration_seconds"] = df["duration_timedelta"].dt.total_seconds()

In [12]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:51,111.0
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:24,84.0
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:56,116.0
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:03:15,195.0


Next task:
- Subset `df` to only include 1v1 battles.

While the majority of the videos posted on the [FlipTop YouTube channel](https://www.youtube.com/@fliptopbattles) are rap battles, a number of them are video flyers, announcement videos, behind-the-scenes content, reaction videos, and the like. As such, it's important to filter out these videos so that we're left to analyze the data from the **rap battles** alone.

### But what constitutes a **rap battle**?

Good question.

My criteria for what is considered a rap battle for the purposes of this project:
1. The video needs to be *a capella* (no underlying beat to accompany emcees rapping).
2. The video involves emcees performing written material (not all of their rounds are off-the-top freestyle). 
    - The early days of FlipTop saw emcees testing each other's skills in the artform known as off-the-top freestyle, where emcees would take turns berating each other lyrically with material they thought of on the spot or in the moment of speaking. 
3. To a lesser extent, the video needs to involve judging at the end (there needs to be stakes).

Note:
- **By these criteria, earlier videos of FlipTop wouldn't be included. Need explanation for what I'm doing here.**

As a long-time viewer of these videos, there's a couple key words that makes filtering with these criteria in mind easier. 
- Include:
    - **vs** - most, if not all, of the a capella rap battles in the FlipTop YouTube channel have "vs" in the video title. 
        - For the uninitiated: "vs" is short for "versus." 
- Exclude:
    - **tryout** - while these are battles, they are for the newcomers to the scene and is often not judged, especially in older videos.
    - **beatbox** - this is another genre of battle separate from the a capella, judged battles.
    - **flyer** and **promo** - these are advertisements and announcement videos for upcoming events.
    - **Anygma Machine** - Anygma, the head of FlipTop as a company, sometimes reviews battles and gives his take on upcoming matches.
        - A reference to the real [Enigma Machine](https://en.wikipedia.org/wiki/Enigma_machine) that the allies had to break in WW2.
   - **[LIVE]** - live performances from the FlipTop Festival event that happened in 2020.
   - **UnggoYan** - Emcees read comments left on videos of their previous battles
   - **Pre-Battle Interviews** - self-explanatory
   - **Salitang Ugat** - translation: "root word." These are interviews of notable emcees who tell the stories behind how they came up with their rap battle name.
   - **Trailer** - promo video trailer for upcomming events
   - **Video Flyer** - self-explanatory
   - **Silip** - BTS videos added recently
   - **Sound Check** - Pre-event check in with FlipTop event prep stuff
   - **Tribute** - tribute to dead rappers
   - **Tutok** - other BTS videos?
   - 

For the scope of this project, I will only consider the battles that are between two people. FlipTop has a variety of rap battle formats, not just two people insulting each other back and forth. Examples include: the Royal Rumble, the 5 vs 5, and the tag-team 2 vs 2 (Dos Por Dos) matches. The vast majority of the battles, though, are one versus one. Those battles will be the focus of this project.

In [13]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:51,111.0
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:24,84.0
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:56,116.0
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:03:15,195.0


In [14]:
df.shape

(1749, 12)

In [15]:
# Include if "vs" in the title
with_vs = df[df["title"].str.contains(r"\bvs\b", case=False, regex=True)]

# Key words and phrases to exclude
exclude_keywords = [
    "tryout", "tryouts", "beatbox", "beat box", "flyer", "promo", "promos", 
    "anygma machine", "unggoyan", "pre-battle interviews", "interview",
    "interviews", "salitang ugat", "trailer", "video flyer", "[live]",
    "silip", "sound check", "tribute", "anniversary party","tutok", "review",
    "abangan"
]

# Regex OR pattern to discard non-battles
exclude_pattern = "|".join([re.escape(word) for word in exclude_keywords])


df_f = with_vs[~with_vs["title"].str.contains(exclude_pattern, case=False, regex=True)]


In [16]:
df_f.shape

(1285, 12)

In [17]:
df_f.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
5,RDiDkJHcnqs,FlipTop - Negho Gy vs Hespero,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-06-25 12:44:16+00:00,334399,PT32M9S,https://www.youtube.com/watch?v=RDiDkJHcnqs,4636,764,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:09,1929.0
6,9xWuT-atbuo,FlipTop - Yagi vs JP,"FlipTop presents: Pakusganay 8 @ Davao City, M...",2025-03-24 12:51:25+00:00,157503,PT38M52S,https://www.youtube.com/watch?v=9xWuT-atbuo,3078,904,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:38:52,2332.0
7,UfFT7TbtRy0,FlipTop - No. 144 vs Markong Bungo,FlipTop presents: Second Sight 13 @ TIU Theate...,2024-12-04 12:45:47+00:00,744384,PT26M,https://www.youtube.com/watch?v=UfFT7TbtRy0,15920,2717,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:26:00,1560.0
8,zWTapWo_Iho,FlipTop - Shaboy vs Marichu,FlipTop presents: Won Minutes Luzon 2 @ 88Fry...,2024-08-03 09:58:17+00:00,294043,PT13M50S,https://www.youtube.com/watch?v=zWTapWo_Iho,4583,689,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:13:50,830.0


I still see non-battles from the titles. Need to inspect all titles from `df_f`.

In [18]:
# Inspection
import os
os.makedirs("../data", exist_ok=True)
df_f[["title"]].to_csv("../data/to_filter.csv", index=False)

Low-hanging fruit: "Abangan si..." or "Abangan sa..." are NOT battles but *clips* from battles.

In [18]:
df_f.shape

(1285, 12)

In [19]:
df_f.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
5,RDiDkJHcnqs,FlipTop - Negho Gy vs Hespero,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-06-25 12:44:16+00:00,334399,PT32M9S,https://www.youtube.com/watch?v=RDiDkJHcnqs,4636,764,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:09,1929.0
6,9xWuT-atbuo,FlipTop - Yagi vs JP,"FlipTop presents: Pakusganay 8 @ Davao City, M...",2025-03-24 12:51:25+00:00,157503,PT38M52S,https://www.youtube.com/watch?v=9xWuT-atbuo,3078,904,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:38:52,2332.0
7,UfFT7TbtRy0,FlipTop - No. 144 vs Markong Bungo,FlipTop presents: Second Sight 13 @ TIU Theate...,2024-12-04 12:45:47+00:00,744384,PT26M,https://www.youtube.com/watch?v=UfFT7TbtRy0,15920,2717,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:26:00,1560.0
8,zWTapWo_Iho,FlipTop - Shaboy vs Marichu,FlipTop presents: Won Minutes Luzon 2 @ 88Fry...,2024-08-03 09:58:17+00:00,294043,PT13M50S,https://www.youtube.com/watch?v=zWTapWo_Iho,4583,689,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:13:50,830.0


Now it's time to include only 1v1 battles.

In [20]:
import re

def is_probably_not_1v1(title):
    if not isinstance(title, str):
        return True  # defensive: non-string, toss it

    title_lower = title.lower()

    # Heuristic 1: multiple "vs"
    if len(re.findall(r"\bvs\b", title_lower)) > 1:
        return True

    # Heuristic 2: "and" appears more than once (i.e., on both sides of the battle)
    if len(re.findall(r"\band\b", title_lower)) > 1:
        return True

    # Heuristic 3: slash (with or without spaces)
    if re.search(r"\s*/\s*|\s*/|\*/\s*", title_lower):
        return True

    # Heuristic 4: plus sign (no-shows)
    if "+" in title_lower:
        return True

    # Heuristic 5: common 2v2 pattern: "and" on both sides of "vs"
    if re.search(r"\band\b.*\bvs\b.*\band\b", title_lower):
        return True
    
    # Heuristic 6: team format like "5 on 5 battle"
    if re.search(r"\b\d+\s*on\s*\d+\b", title_lower):
        return True

    return False


# Filtering
mask = df_f["title"].apply(is_probably_not_1v1)
df_not_1v1 = df_f[mask]         # Filtered out (bad) - inspect
df_1v1_only = df_f[~mask]       # Final 1v1 dataset? Pending inspection
    


In [21]:
# Audit output
print("Filtered out (not 1v1):")
for title in df_not_1v1["title"]:
    print("-", title)

Filtered out (not 1v1):
- FlipTop - Crazymix/Bassilyo vs Cripli/Towpher @ DosPorDos 2017
- FlipTop - Rapido/Icaruz vs Dopee/Snob @ Dos Por Dos 2
- FlipTop - CripLi / Towpher vs K-Ram / SlockOne
- FlipTop - Caspher / Hespero vs Atoms / Cygnus @ DosPorDos 2024 Finals
- FlipTop - Negho G / Pamoso vs Atoms / Cygnus @ DosPorDos 2024 Semifinals
- FlipTop - Caspher / Hespero vs Aubrey / Marichu @ DosPorDos 2024 Semifinals
- FlipTop - Negho Gy / Pamoso vs RG / Deadline @ DosPorDos2024
- FlipTop - Aubrey / Marichu vs Sickreto / Article Clipted @ DosPorDos2024
- FlipTop - Bisente / Jawz vs Atoms / Cygnus @ DosPorDos 2024
- FlipTop - Caspher / Hespero vs Kenzer / Mimack @ DosPorDos 2024
- FlipTop - Frooz / Elbiz vs Mac T / G-Spot
- FlipTop - Pistolero / Luxuria vs MastaFeat / Hearty | Surprise Freestyle Battle
- FlipTop - Sur Henyo vs Kregga vs Batang Rebelde vs LilStrocks vs Bagsik *ROYAL RUMBLE*
- FlipTop - K-Ram/SlockOne vs Vitrum/Illtimate
- FlipTop - Zaito vs C-Quence vs CNine vs Prince Rhym

In [22]:
df_not_1v1[["title"]].to_csv("../data/not_1v1_filtered_out.csv", index=False)

In [23]:
df_1v1_only.head(3)

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
5,RDiDkJHcnqs,FlipTop - Negho Gy vs Hespero,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-06-25 12:44:16+00:00,334399,PT32M9S,https://www.youtube.com/watch?v=RDiDkJHcnqs,4636,764,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:09,1929.0
6,9xWuT-atbuo,FlipTop - Yagi vs JP,"FlipTop presents: Pakusganay 8 @ Davao City, M...",2025-03-24 12:51:25+00:00,157503,PT38M52S,https://www.youtube.com/watch?v=9xWuT-atbuo,3078,904,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:38:52,2332.0


In [24]:
df_1v1_only.shape

(1194, 12)

In [25]:
df_1v1_only.loc[:, "title"] = df_1v1_only["title"].str.strip()

In [26]:
# Strip leading and trailing " from title — only if they exist
df_1v1_only = df_1v1_only.copy()
df_1v1_only["title"] = df_1v1_only["title"].str.replace(r'^"(.*)"$', r'\1', regex=True)

In [27]:
df_1v1_only.shape

(1194, 12)

Need to inspect this 1v1 only df to see if it still contains non-1v1 battles

In [28]:
df_1v1_only[["title"]].to_csv("../data/filtered_1v1.tsv", sep="\t", index=False)

There are commas in 3 of the videos that are 1v1 battles. Writing to csv without specifying tab separated values instead puts "quotes" on those battles.
- Solved.

Looks all good on cursory inspection.

Next task: deal with early FlipTop battles having parts `pt. 1` and `pt. 2` etc.
- Make it into 1 video but now the links are a list instead of only string?

In [29]:
df_1v1_only.columns

Index(['id', 'title', 'description', 'upload_date', 'view_count', 'duration',
       'url', 'likeCount', 'commentCount', 'tags', 'duration_timedelta',
       'duration_seconds'],
      dtype='object')

In [30]:
df_1v1_only.dtypes

id                                 object
title                              object
description                        object
upload_date           datetime64[ns, UTC]
view_count                         object
duration                           object
url                                object
likeCount                          object
commentCount                       object
tags                               object
duration_timedelta        timedelta64[ns]
duration_seconds                  float64
dtype: object

In [31]:
df_1v1_only["view_count"] = pd.to_numeric(df_1v1_only["view_count"], errors="coerce")
df_1v1_only["likeCount"] = pd.to_numeric(df_1v1_only["likeCount"], errors="coerce")
df_1v1_only["commentCount"] = pd.to_numeric(df_1v1_only["commentCount"], errors="coerce")

In [32]:
df_1v1_only.head(5)

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
5,RDiDkJHcnqs,FlipTop - Negho Gy vs Hespero,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-06-25 12:44:16+00:00,334399,PT32M9S,https://www.youtube.com/watch?v=RDiDkJHcnqs,4636,764,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:09,1929.0
6,9xWuT-atbuo,FlipTop - Yagi vs JP,"FlipTop presents: Pakusganay 8 @ Davao City, M...",2025-03-24 12:51:25+00:00,157503,PT38M52S,https://www.youtube.com/watch?v=9xWuT-atbuo,3078,904,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:38:52,2332.0
7,UfFT7TbtRy0,FlipTop - No. 144 vs Markong Bungo,FlipTop presents: Second Sight 13 @ TIU Theate...,2024-12-04 12:45:47+00:00,744384,PT26M,https://www.youtube.com/watch?v=UfFT7TbtRy0,15920,2717,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:26:00,1560.0
8,zWTapWo_Iho,FlipTop - Shaboy vs Marichu,FlipTop presents: Won Minutes Luzon 2 @ 88Fry...,2024-08-03 09:58:17+00:00,294043,PT13M50S,https://www.youtube.com/watch?v=zWTapWo_Iho,4583,689,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:13:50,830.0


In [33]:
with_parts = df_1v1_only.copy()

In [35]:
import pandas as pd
import re

def consolidate_battle_parts(df):
    """
    Consolidates FlipTop battles split into pt. 1 and pt. 2 into a single row.
    Keeps Part 1's description and metadata, sums counts, and merges URLs.
    """
    df = df.copy()

    # Convert relevant columns to numeric just in case
    df["view_count"] = pd.to_numeric(df["view_count"], errors="coerce").fillna(0).astype(int)
    df["likeCount"] = pd.to_numeric(df["likeCount"], errors="coerce").fillna(0).astype(int)
    df["commentCount"] = pd.to_numeric(df["commentCount"], errors="coerce").fillna(0).astype(int)

    # Helper to normalize titles to identify pairs
    def get_base_title(title):
        return re.sub(r'\s*pt\.?\s*[12]$', '', title.strip(), flags=re.IGNORECASE).strip()

    def seconds_to_iso8601_duration(seconds):
        seconds = int(round(seconds))  # Ensure integer
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        secs = seconds % 60

        duration_str = "PT"
        if hours:
            duration_str += f"{hours}H"
        if minutes:
            duration_str += f"{minutes}M"
        if secs or (not hours and not minutes):  # Avoid empty PT
            duration_str += f"{secs}S"
        return duration_str

    # Identify rows with "pt. 1" or "pt. 2"
    is_part = df["title"].str.contains(r'\spt\.?\s*[12]$', case=False, regex=True)
    part_df = df[is_part].copy()
    part_df["base_title"] = part_df["title"].apply(get_base_title)

    grouped = part_df.groupby("base_title")

    consolidated_rows = []
    seen_titles = set()

    for base_title, group in grouped:
        if len(group) != 2:
            continue  # Skip if not exactly two parts

        # Ensure pt. 1 comes first
        sorted_group = group.sort_values("title")
        part1 = sorted_group.iloc[0]
        part2 = sorted_group.iloc[1]

        total_seconds = part1["duration_seconds"] + part2["duration_seconds"]

        consolidated = {
            "title": base_title,
            "description": part1["description"],
            "upload_date": part1["upload_date"],
            "url": [part1["url"], part2["url"]],
            "view_count": part1["view_count"] + part2["view_count"],
            "likeCount": part1["likeCount"] + part2["likeCount"],
            "commentCount": part1["commentCount"] + part2["commentCount"],
            "tags": list(set((part1["tags"] or []) + (part2["tags"] or []))),
            "duration": seconds_to_iso8601_duration(total_seconds),
            "duration_seconds": total_seconds,
            "duration_timedelta": pd.to_timedelta(total_seconds, unit="s"),
            "id": [part1["id"], part2["id"]],
        }
        consolidated_rows.append(consolidated)
        seen_titles.update(group.index)

    # Build consolidated DataFrame
    consolidated_df = pd.DataFrame(consolidated_rows)

    # Keep all other rows (not part of pt. 1/pt. 2)
    remaining_df = df.drop(index=seen_titles)

    # Combine and return
    final_df = pd.concat([remaining_df, consolidated_df], ignore_index=True)
    return final_df


In [36]:
df_cleaned = consolidate_battle_parts(with_parts)

In [37]:
df_cleaned.columns

Index(['id', 'title', 'description', 'upload_date', 'view_count', 'duration',
       'url', 'likeCount', 'commentCount', 'tags', 'duration_timedelta',
       'duration_seconds'],
      dtype='object')

In [38]:
df_cleaned[["id", "title", "view_count", "duration", "upload_date", "description"]].to_csv("../data/inspect_df_cleaned.tsv", sep="\t", index=True)

In [39]:
df_cleaned.tail(20)

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
1163,esPXg5uQIgI,FlipTop - Plazma vs Tim,"FlipTop presents: Ahon @ Guerrilla Radio, Pasi...",2010-09-02 12:49:23+00:00,514798,PT14M5S,https://www.youtube.com/watch?v=esPXg5uQIgI,593,903,"[plazma, vs, tim, fliptop, FlipTop, philippine...",0 days 00:14:05,845.0
1164,EynkMnr4pfc,FlipTop - Ghandi vs Sev,"FlipTop presents: Ahon @ Guerrilla Radio, Pasi...",2010-08-23 17:29:50+00:00,342285,PT8M23S,https://www.youtube.com/watch?v=EynkMnr4pfc,230,485,"[ghandi, sev, moment, squad, kalayaan, smc, fl...",0 days 00:08:23,503.0
1165,HYXGRNluj-o,FlipTop - Apekz vs Harlem (Freestyle),"FlipTop presents: Second Sight @ B-Side, Makat...",2010-08-21 08:42:09+00:00,1237254,PT10M24S,https://www.youtube.com/watch?v=HYXGRNluj-o,1360,996,"[APekz, vs, harlem, schizophrenia, valcity, va...",0 days 00:10:24,624.0
1166,rGMcIGfnTL4,FlipTop - Tim vs DimeBag Daryl (Edited),"FlipTop presents: Second Sight @ B-Side, Makat...",2010-08-20 14:24:10+00:00,192987,PT5M11S,https://www.youtube.com/watch?v=rGMcIGfnTL4,144,172,"[Tim, Dimebag, daryl, fliptop, filipino, hipho...",0 days 00:05:11,311.0
1167,IyH87QWShus,FlipTop - Batas vs Abra,"***RESURRECTED FOOTAGE***\nSHOUTS TO JEDLI, RB...",2010-06-11 18:47:32+00:00,1405113,PT8M29S,https://www.youtube.com/watch?v=IyH87QWShus,999,727,"[abra, lyrically, deranged, poets, LDP, batas,...",0 days 00:08:29,509.0
1168,2A84cY0Ehu0,FlipTop - JedLi vs Apoc,"***RESURRECTED FOOTAGE***\nSHOUTS TO JEDLI, RB...",2010-06-11 18:02:17+00:00,195724,PT8M36S,https://www.youtube.com/watch?v=2A84cY0Ehu0,100,29,"[jedli, apoc, people's, future, mr., vito, cre...",0 days 00:08:36,516.0
1169,qQYa5kkggXg,FlipTop - Plazma vs Apoc,"FlipTop presents: Second Sight @ B-Side, Makat...",2010-05-04 09:48:43+00:00,941408,PT9M2S,https://www.youtube.com/watch?v=qQYa5kkggXg,748,704,"[FLIPTOP, FlipTop, Plazma, Apoc, AMPON, heaven...",0 days 00:09:02,542.0
1170,Y4l-HpCaHmQ,FlipTop - Target vs Luke,"FlipTop presents: Second Sight @ B-Side, Makat...",2010-04-21 15:36:02+00:00,3026820,PT9M13S,https://www.youtube.com/watch?v=Y4l-HpCaHmQ,1580,2262,"[FLIPTOP2, target, luke, 720P, b-side, makati,...",0 days 00:09:13,553.0
1171,g_5D-FOK75Y,FlipTop - Cameltoe vs Dhictah,"FlipTop presents: Second Sight @ B-Side, Makat...",2010-04-20 19:47:29+00:00,1553456,PT10M9S,https://www.youtube.com/watch?v=g_5D-FOK75Y,1185,1117,"[cameltoe, dhictah, fliptop, FlipTop, Battle, ...",0 days 00:10:09,609.0
1172,"[Tycr21-NO5s, c4W2A3GdtzM]",FlipTop - Batas vs Fuego,"FlipTop presents: Ahon @ Guerrilla Radio, Pasi...",2010-07-13 03:16:55+00:00,7506575,PT15M24S,"[https://www.youtube.com/watch?v=Tycr21-NO5s, ...",8341,6038,"[emerson, radio, fliptop battle, league, kingo...",0 days 00:15:24,924.0


In [40]:
df_cleaned.dtypes

id                                 object
title                              object
description                        object
upload_date           datetime64[ns, UTC]
view_count                          int64
duration                           object
url                                object
likeCount                           int64
commentCount                        int64
tags                               object
duration_timedelta        timedelta64[ns]
duration_seconds                  float64
dtype: object

In [41]:
df_cleaned[["title"]].to_csv("../data/inspect_titles.tsv", sep="\t", index=False)

Looks good upon inspection. Battles with parts consolidated!

Next task: 
- Add matchup column e.g. Emcee1 vs Emcee2
- Add emcee1 column e.g. Loonie
- Add emcee2 column e.g. Tipsy D

In [42]:
df_cleaned.head(15)

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,RDiDkJHcnqs,FlipTop - Negho Gy vs Hespero,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-06-25 12:44:16+00:00,334399,PT32M9S,https://www.youtube.com/watch?v=RDiDkJHcnqs,4636,764,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:09,1929.0
2,9xWuT-atbuo,FlipTop - Yagi vs JP,"FlipTop presents: Pakusganay 8 @ Davao City, M...",2025-03-24 12:51:25+00:00,157503,PT38M52S,https://www.youtube.com/watch?v=9xWuT-atbuo,3078,904,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:38:52,2332.0
3,UfFT7TbtRy0,FlipTop - No. 144 vs Markong Bungo,FlipTop presents: Second Sight 13 @ TIU Theate...,2024-12-04 12:45:47+00:00,744384,PT26M,https://www.youtube.com/watch?v=UfFT7TbtRy0,15920,2717,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:26:00,1560.0
4,zWTapWo_Iho,FlipTop - Shaboy vs Marichu,FlipTop presents: Won Minutes Luzon 2 @ 88Fry...,2024-08-03 09:58:17+00:00,294043,PT13M50S,https://www.youtube.com/watch?v=zWTapWo_Iho,4583,689,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:13:50,830.0
5,tSn0rQCC7uM,FlipTop - Sickreto vs Murdz,"FlipTop presents: Ahon 14, Day 2 @ TIU Theater...",2024-04-04 12:33:45+00:00,162762,PT29M24S,https://www.youtube.com/watch?v=tSn0rQCC7uM,2747,757,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:29:24,1764.0
6,tsEtOkvu4HM,FlipTop - Empithri vs Murdz,"FlipTop presents: Gubat 12, Day 1 @ Mariner's ...",2023-10-31 12:36:28+00:00,268396,PT28M30S,https://www.youtube.com/watch?v=tsEtOkvu4HM,3804,564,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:28:30,1710.0
7,LOzAhN92lo4,FlipTop - Pen Pluma vs GI,FlipTop presents: Gubat 11 @ Mandaue Sports Co...,2023-05-26 12:51:12+00:00,481978,PT28M48S,https://www.youtube.com/watch?v=LOzAhN92lo4,5700,803,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:28:48,1728.0
8,Y0hgfpGo3cE,FlipTop - Lhipkram vs J-King,"FlipTop presents: Zoning 15 @ TIU Theater, Mak...",2022-11-30 12:30:26+00:00,2542323,PT33M11S,https://www.youtube.com/watch?v=Y0hgfpGo3cE,35452,2707,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:33:11,1991.0
9,MfH1NaCu7Ps,FlipTop - Poison13 vs SlockOne @ Isabuhay 2022,FlipTop presents: Second Sight 10 @ TIU Theate...,2022-05-26 12:58:20+00:00,2384593,PT32M1S,https://www.youtube.com/watch?v=MfH1NaCu7Ps,39388,2277,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:01,1921.0


In [43]:
def extract_matchup(title):
    match = re.search(r'-\s*(.*?)\s+vs\s+(.*?)(?:\s+@|\s+\(|$)', title, re.IGNORECASE)
    if match:
        emcee1 = match.group(1).strip()
        emcee2 = match.group(2).strip()
        matchup = f"{emcee1} vs {emcee2}"
        return matchup, emcee1, emcee2
    return None, None, None

In [52]:
with_matches = df_cleaned.copy()

In [49]:
emcees_df = pd.read_csv("../data/emcees.csv")

In [50]:
emcee_names = set(emcees_df["name"].dropna().str.strip().str.lower())

In [51]:
def extract_matchup(title, emcee_names):
    title = title.strip()

    # Remove "FlipTop" prefix with optional extras and dash
    title = re.sub(r'^FlipTop(?: [^–-]+)?\s*[-–]\s*', '', title, flags=re.IGNORECASE)

    # Look for "vs" as delimiter
    if " vs " not in title.lower():
        return None

    # Split once at "vs"
    parts = re.split(r'\s+vs\s+', title, maxsplit=1, flags=re.IGNORECASE)
    if len(parts) != 2:
        return None

    emcee1 = parts[0].strip()
    
    # Now clean emcee2: remove any suffix that begins with @, |, (, *
    emcee2_raw = parts[1].strip()
    emcee2_clean = re.split(r'\s*[@|(*]', emcee2_raw)[0].strip()

    # Optionally validate both emcees exist in list
    if emcee1.lower() in emcee_names and emcee2_clean.lower() in emcee_names:
        return f"{emcee1} vs {emcee2_clean}"
    
    # If emcee name check fails, still return the cleaned guess
    return f"{emcee1} vs {emcee2_clean}"

In [54]:
with_matches["matchup"] = with_matches["title"].apply(lambda t: extract_matchup(t, emcee_names))

In [55]:
with_matches.head(5)

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds,matchup
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0,Caspher vs CRhyme
1,RDiDkJHcnqs,FlipTop - Negho Gy vs Hespero,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-06-25 12:44:16+00:00,334399,PT32M9S,https://www.youtube.com/watch?v=RDiDkJHcnqs,4636,764,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:09,1929.0,Negho Gy vs Hespero
2,9xWuT-atbuo,FlipTop - Yagi vs JP,"FlipTop presents: Pakusganay 8 @ Davao City, M...",2025-03-24 12:51:25+00:00,157503,PT38M52S,https://www.youtube.com/watch?v=9xWuT-atbuo,3078,904,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:38:52,2332.0,Yagi vs JP
3,UfFT7TbtRy0,FlipTop - No. 144 vs Markong Bungo,FlipTop presents: Second Sight 13 @ TIU Theate...,2024-12-04 12:45:47+00:00,744384,PT26M,https://www.youtube.com/watch?v=UfFT7TbtRy0,15920,2717,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:26:00,1560.0,No. 144 vs Markong Bungo
4,zWTapWo_Iho,FlipTop - Shaboy vs Marichu,FlipTop presents: Won Minutes Luzon 2 @ 88Fry...,2024-08-03 09:58:17+00:00,294043,PT13M50S,https://www.youtube.com/watch?v=zWTapWo_Iho,4583,689,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:13:50,830.0,Shaboy vs Marichu


In [56]:
with_matches[["title", "matchup"]].to_csv("../data/inspect_matchups.tsv", sep="\t", index=False)

In [57]:
# Manual post-processing
with_matches["matchup"] = with_matches["matchup"].str.replace(r'\s-\s.*$', '', regex=True)

In [59]:
# Just in case there's still any leading of trailing white space
with_matches["matchup"] = with_matches["matchup"].str.strip()

In [60]:
with_matches[["title", "matchup"]].to_csv("../data/inspect_matchups.tsv", sep="\t", index=False)

Next task: `emcee1` and `emcee2` column from `matchup` column. Or maybe standardize the names of the emcees first using `emcees.csv` that we got from FlipTop's website.